In [1]:
from pynq import Overlay

In [2]:
!ls /home/xilinx/pynq/overlays/

adder	     dma_sequel    fir_accel3	matmul1  matmul6	__pycache__
add_vec      dma_sequel_2  __init__.py	matmul3  move_ab	sigmoid
base	     fir_accel	   linreg_vadd	matmul4  new_adder	sigmoid2
dma_prequel  fir_accel2    logictools	matmul5  new_fir_accel


In [3]:
!ls /home/xilinx/pynq/overlays/sigmoid2

sigmoid.bit  sigmoid.hwh  sigmoid.tcl


In [4]:
ol = Overlay("/home/xilinx/pynq/overlays/sigmoid2/sigmoid.bit")

In [5]:
help(ol)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay /home/xilinx/pynq/overlays/sigmoid2/sigmoid.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    toplevelsigmoid_0    : pynq.overlay.DefaultIP
    processing_system7_0 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [6]:
tl = ol.toplevelsigmoid_0

In [7]:
tl.register_map

RegisterMap {
  a = Register(a=write-only),
  b = Register(b=1056964608),
  b_ctrl = Register(b_ap_vld=1, RESERVED=0)
}

In [8]:
# tl.write(0x10, 0.1)

# cannot simply write float, have to pack it in bytes
# from pynq import allocate
# a = allocate(shape=(1,), dtype=np.float32)
# a[0] = 0.1

In [9]:
import struct

In [27]:
a = -0.5
a_data = struct.unpack("!I", struct.pack("!f", a))[0]

In [28]:
a_data, a

(3204448256, -0.5)

In [29]:
tl.write(0x10, a_data)

In [30]:
res = tl.read(0x18)

In [31]:
res_float = struct.unpack("!f", struct.pack("!I", res))[0]

In [32]:
res_float

0.4018288850784302

In [33]:
import numpy as np

In [34]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [35]:
sigmoid(a)

0.3775406687981454

it works!!!

lets try a different value

In [36]:
a = 1
a_data = struct.unpack("!I", struct.pack("!f", a))[0]

In [37]:
a_data, a

(1065353216, 1)

In [38]:
tl.write(0x10, a_data)

In [39]:
res = tl.read(0x18)

In [40]:
res_float = struct.unpack("!f", struct.pack("!I", res))[0]

In [41]:
res_float

0.6930490136146545

In [42]:
sigmoid(a)

0.7310585786300049

quite close!

# OLD

## send to board

In [ ]:
from pynq import allocate
import numpy as np

data_size = len(m2_to_board)
input_buffer = allocate(shape=(data_size,), dtype=np.int32)

In [ ]:
print("py_buffer physical address {}".format(hex(input_buffer.physical_address)))

In [ ]:
out_size = len(m3.ravel())
output_buffer = allocate(shape=(out_size,), dtype=np.int32)

In [ ]:
print("py_buffer physical address {}".format(hex(output_buffer.physical_address)))

In [ ]:
tl = ol.toplevelmatmul_0

In [ ]:
tl.register_map

In [ ]:
tl.register_map.in_mat_1 = input_buffer.physical_address
tl.register_map.result_1 = output_buffer.physical_address

## Start the IP

In [ ]:
tl.register_map.CTRL.AP_START = 1

If we start the IP once, when the IP completes an iteration DONE will go high once. While the IP is working, AP_IDLE will be 0. In this example, the time for an iteration of the HLS IP is very low so you will probably not see AP_IDLE = 0.

Done is cleared when it is read. There is a limitation with the register_map. Multiple reads to the same register may be made to collect the information required for the register_map. This may mean DONE is cleared while the values of some of the other bits and registers are being read and consequentally will not show as high in the register m

In [ ]:
tl.register_map.CTRL.AP_DONE

In [ ]:
tl.register_map.CTRL.AP_IDLE

In [ ]:
output_buffer

# With DMA

In [ ]:
dma = ol.axi_dma_0

In [ ]:
help(dma)

In [ ]:
dma.buffer_max_size

In [ ]:
e = ol.toplevelmatmul_0

In [ ]:
help(e)

In [ ]:
e.register_map

## Create aliases

In [ ]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

hls_ip = ol.toplevelmatmul_0

In [ ]:
hls_ip.register_map

Note that the HLS IP is not started yet (AP_START=0). You can also see the IP is idle (AP_IDLE=1).

We will start the HLS IP and then start some transfers from the DMA.

We could initiate the DMA transfers first if we preferred. The DMA transfers would stall until the IP is started.


## Start the HLS IP

We can start the HLS IP by writing 0x81 to the control register. This will set bit 0 (AP_START) to “1” and bit 7 (AUTO_RESTART) to “1”. AUTO_RESTART means the IP will run continuously. If we don’t set this then after the IP completes one full operation or iteration, it will stop and wait until AP_START is set again. We would have to set this every time we want the IP to process some data.

In [ ]:
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81)  # 0x81 will set bit 0

Check the correct bits have been set.



In [ ]:
hls_ip.register_map

## numpy calculate

In [ ]:
import numpy as np

In [ ]:
# multiply m1 by m2
m1 = np.zeros(shape=(6, 5))
m2 = np.zeros(shape=(5, 4))
for i in range(6):
    for j in range(5):
        m1[i, j] = i * 5 + j

for i in range(5):
    for j in range(4):
        m2[i, j] = i * 4 + j + 1

the matrix that is on the board:

In [ ]:
res = []
for r in m1:
    res.append(", ".join([f"{x:.0f}" for x in r]))
print(",\n".join(res))

the matrix that we will send to the board:

In [ ]:
res = []
for r in m2:
    res.append(", ".join([f"{x:.0f}" for x in r]))
print(",\n".join(res))

the result

In [ ]:
m3 = np.matmul(m1, m2)

In [ ]:
res = []
for r in m3:
    res.append(", ".join([f"{x:.0f}" for x in r]))
print(",\n".join(res))

### flatten the matrix

In [ ]:
m2_to_board = m2.ravel()
m2_to_board = m2_to_board.astype(np.int32)

In [ ]:
m2_to_board

## DMA send

In [ ]:
from pynq import allocate
import numpy as np

data_size = len(m2_to_board)
input_buffer = allocate(shape=(data_size,), dtype=np.int32)

In [ ]:
for i in range(data_size):
    input_buffer[i] = m2_to_board[i]

In [ ]:
input_buffer

In [ ]:
dma_send.transfer(input_buffer)

In [ ]:
dma_send.idle

In [ ]:
dma_recv.idle

## DMA receive

Readback data from the HLS IP and store in DRAM. Start by creating the output buffer



In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.int32)

In [ ]:
dma_recv.transfer(output_buffer)

In [ ]:
output_buffer

In [ ]:
dma_recv.error

In [ ]:
dma_recv.idle

In [ ]:
dma_recv.transferred

In [ ]:
hls_ip.register_map

In [ ]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer - 5)))

In [ ]:
input_buffer

In [ ]:
output_buffer

problem: we are transferring 64 bits => notice that 0 goes to 5, but the second number is 7 (2+5)

In [ ]:
hls_ip.register_map

In [ ]:
del input_buffer, output_buffer

In [ ]:
ol.free()

# OLD

In [ ]:
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

In [ ]:
from pynq import allocate
import numpy as np

data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [ ]:
for i in range(data_size):
    input_buffer[i] = i + 0xCAFE0000

Let’s check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

In [ ]:
for i in range(10):
    print(hex(input_buffer[i]))

Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.



In [ ]:
dma_send.transfer(input_buffer)

Let’s read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.



In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print("0x" + format(output_buffer[i], "02x"))

In [ ]:
dma_recv.transfer(output_buffer)

In [ ]:
for i in range(10):
    print("0x" + format(output_buffer[i], "02x"))

In [ ]:
print("Arrays are equal: {}".format(np.array_equal(input_buffer, output_buffer)))

In [ ]:
input_buffer

In [ ]:
output_buffer

## Check DMA status, and trigger an error


In [ ]:
dma_recv.error

In [ ]:
dma_recv.idle

In [ ]:
dma_recv.transfer(output_buffer)

In [ ]:
dma_recv.idle

Start another receive transfer while the DMA is not idle



In [ ]:
dma_recv.transfer(output_buffer)

We can check the running state of the DMA



In [ ]:
dma_recv.running

In [ ]:
dma_recv.idle

In [ ]:
dma_send.idle

## Check the DMA register map


We can read back individual status bits as show above. It can be useful to read back the full register map which will give details on all control and status bits. The meaning of each register and each bit will not be covered. For more details you can refer to the product guide for the DMA.



In [ ]:
dma.register_map

In [ ]:
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("---")
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

## Free all memory

In [ ]:
del input_buffer, output_buffer

In [ ]:
ol.free()

In [ ]:
help(ol.free)

In [ ]:
help(ol)

# OLD

In [ ]:
tl = overlay.toplevel_0
help(tl)

In [ ]:
tl.register_map

In [ ]:
from pynq import allocate
import numpy as np

py_buffer = allocate(shape=(100,), dtype=np.int32)

In [ ]:
print("py_buffer physical address {}".format(hex(py_buffer.physical_address)))

## Configure the IP

Write the values of the memory address, the length, and the value to be added to each location. You must make sure that length doesn't exceed the lendth of the memory that we allocated earlier AND the maximum value that can be set in the HLS IP. The buffe

In [ ]:
tl.register_map.a_1 = py_buffer.physical_address
tl.register_map.length_r = 100
tl.register_map.value_r = 7

## Start the IP

In [ ]:
tl.register_map.CTRL.AP_START = 1

If we start the IP once, when the IP completes an iteration DONE will go high once. While the IP is working, AP_IDLE will be 0. In this example, the time for an iteration of the HLS IP is very low so you will probably not see AP_IDLE = 0.

Done is cleared when it is read. There is a limitation with the register_map. Multiple reads to the same register may be made to collect the information required for the register_map. This may mean DONE is cleared while the values of some of the other bits and registers are being read and consequentally will not show as high in the register m

In [ ]:
tl.register_map.CTRL.AP_DONE

In [ ]:
tl.register_map.CTRL.AP_IDLE

If you have a long running HLS IP, you can poll on the DONE bit before continuing with your application.

## Check the contents of the memory

In [ ]:
py_buffer

In [ ]:
len(py_buffer)

In [ ]:
my_ip.register_map.CTRL.AUTO_RESTART = 1

In [ ]:
# %xdel in1_vadd
# %xdel in2_vadd
# %xdel out
overlay.free()

# OLD

In [ ]:
overlay.scalar_add.add(15, 20)

In [ ]:
add_ip.write(0x10, 4)
add_ip.write(0x18, 5)
add_ip.read(0x20)

In [ ]:
from pynq import DefaultIP


class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ["xilinx.com:hls:add:1.0"]

    def add(self, a, b):
        self.write(0x10, a)
        self.write(0x18, b)
        return self.read(0x20)

In [ ]:
overlay = Overlay("/home/xilinx/pynq/overlays/adder/adder.bit")

In [ ]:
help(overlay)